### Почта для сдачи решений:  
    bdt-mf-ml-nlp-2020-q4@bigdatateam.org

**В теме письма укажите:**
    
    "HW2:CompLing. ФИО"

### Пожалуйста, оцените качество заданий второго модуля [здесь](http://rebrand.ly/mfnlp2020q4_feedback_hw02)
-----------------------------------------------------------------------------------------------------------------

# Dependency Parsing

Мы будем использовать **POS tagging** и **dependency parsing** из популярного пакета [SpaCy](https://spacy.io)

В SpaCy используется подход Universal Dependencies

## Установим модель SpaCy

In [3]:
!pip install spacy

In [6]:
!python -m spacy download en

В SpaCy есть возможность не только размечать части речи и синтаксическиезависимости, но и красиво визуализировать, используя ``displacy``

In [7]:
import spacy
from spacy import displacy
nlp = spacy.load('en')
test_doc = nlp('This is a test.')
displacy.render(test_doc, jupyter = True)

### Spacy сама токенизирует текст, все токены становятся специальными элементами, в которых хранятся сам токен,а также его таги

In [8]:
print([(token, token.pos_) for token in test_doc])

[(This, 'DET'), (is, 'AUX'), (a, 'DET'), (test, 'NOUN'), (., 'PUNCT')]


In [9]:
print(test_doc)

This is a test.


### Для задания мы используем известный корпус литературных произведений - Project Gutenberg. 
Из корпуса мы возьмем довольно длинный роман Джейн Остин "Чувство и чувствительность" (Sense and Sensibility)

In [10]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
sentences = gutenberg.sents('austen-sense.txt')

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/antonina.goryacheva/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


In [11]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

### Вопросы

**1. Сколько предложений в этом романе? Сколько уникальных токенов?**

In [13]:
# предложений
len(sentences)

4999

In [14]:
# уникальных тоокенов
len(set([word.lower() for text in sentences for word in text]))

6398

**2. Найдите 5 наиболее частотных глагольных словоформ? Найдите 5 наиболее частотных лемм (начальных форм) глаголов? (глагол-verb)**

In [17]:
' '.join(sentences[3])

'Their estate was large , and their residence was at Norland Park , in the centre of their property , where , for many generations , they had lived in so respectable a manner as to engage the general good opinion of their surrounding acquaintance .'

In [18]:
test_doc = nlp(' '.join(sentences[3]))

In [ ]:
# глагольные словоформы - это?
# сделать лемматизацию

In [20]:
[(token, token.pos_) for token in test_doc]

[(Their, 'DET'),
 (estate, 'NOUN'),
 (was, 'AUX'),
 (large, 'ADJ'),
 (,, 'PUNCT'),
 (and, 'CCONJ'),
 (their, 'DET'),
 (residence, 'NOUN'),
 (was, 'AUX'),
 (at, 'ADP'),
 (Norland, 'PROPN'),
 (Park, 'PROPN'),
 (,, 'PUNCT'),
 (in, 'ADP'),
 (the, 'DET'),
 (centre, 'NOUN'),
 (of, 'ADP'),
 (their, 'DET'),
 (property, 'NOUN'),
 (,, 'PUNCT'),
 (where, 'ADV'),
 (,, 'PUNCT'),
 (for, 'ADP'),
 (many, 'ADJ'),
 (generations, 'NOUN'),
 (,, 'PUNCT'),
 (they, 'PRON'),
 (had, 'AUX'),
 (lived, 'VERB'),
 (in, 'ADP'),
 (so, 'ADV'),
 (respectable, 'ADJ'),
 (a, 'DET'),
 (manner, 'NOUN'),
 (as, 'SCONJ'),
 (to, 'PART'),
 (engage, 'VERB'),
 (the, 'DET'),
 (general, 'ADJ'),
 (good, 'ADJ'),
 (opinion, 'NOUN'),
 (of, 'ADP'),
 (their, 'DET'),
 (surrounding, 'VERB'),
 (acquaintance, 'NOUN'),
 (., 'PUNCT')]

## Грамматика зависимостей (Dependency parsing) and PP attachment

Теперь посмотрим как глаголы и предложные фразы (prepositional phrase или PP) могут быть связаны в предложении.
- Вершина *предложной фразы* - предлог. Например, "**in** the house", "**on** the table", "**with** my friend"


### Вопросы

  3. В чем отличие в зависимости предложных фраз между предложениями группы (A) и предложениями группы (B)? 
  Посмотрите на схему зависимостей, которую можно сделать с помощью SpaCy.
  - Подсказка: какие "родительские" вершины у предложных групп?
  
(A)
  * I eat an apple in my room.
  * We listen to music at the theater.
  * John visited Brazil with his friend.
  
(B)
  * I see a fly in my soup.
  * She knows the man at the store.
  * I photographed a man with a bowtie.
  

### Бывают случаи, когда не очевидно, к какому элементу предложения нужно присоединить предложную фразу
- Вспомните пример из лекции "I shot an elephant in my pyjamas"

На примере нашего корпуса попробуем понять, какую "родительскую" вершину для предложных фраз предпочитает SpaCy


**### Вопросы:**

  4. Выберите все случаи, когда предложная фраза присоединяется к глаголу (verb), как в группе предложений (A) выше.
      - Создайте список для всех пар (tupples) вида (лемматизированный глагол, лемматизированный предлог) = (verb lemma, preposition lemma).
      - Посчитайте сколько таких случаев.
      - Выпишите 5 наиболее частотный пар (verb lemma, preposition lemma)
      - Подсказка: В spaCy можно получить зависимый токен  с помощью *token*.children, а тип зависимости с помощью *child.dep_*. 
  
  5. Сделайте тоже самое для случаев, когда предложная фраза присоединяется к дополнению глагола (verb's object), как в группе предложений (B). 
      - Создайте список для всех пар (tupples) вида (лемматизированный дополнение, лемматизированный предлог) = (verb's object lemma, preposition lemma).
      - Посчитайте сколько таких случаев.
      - Выпишите 5 наиболее частотный пар (verb's object, preposition lemma)?
  ?
  6. Постройте систему зависимостей для предложения "I shot an elephant in my pyjamas"
      - К чему присоединена предложная группа?
      - Кто был в пижаме с точки зрения SpaCy?

**Бонус:** Выберите несколько случайных предложений из корпуса. Постройте для них систему зависимостей. 
- Похоже на правду?

Решение присылйте на почту  bdt-mf-ml-nlp-2020-q4@bigdatateam.org  

В теме письма укажите: ``HW2:CompLing. ФИО``

Пожалуйста, оставьте обратную связь о задании [по ссылке](http://rebrand.ly/mfnlp2020q4_feedback_hw02). Она (при желании) анонимна ;)